In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date,timedelta

import warnings
warnings.filterwarnings("ignore")

In [2]:
days_to_fix=60 #blank data
days_to_predict=30   #future
prediction_ma_window=3  #prediction ma
ads_ma=7

In [3]:
df=pd.read_csv("60day_roas.csv")
df

,Year,Month,Week,Day,Country,Channel,OS name,Campaign,Clicks,CTR,...,Impressions (paid),Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),Revenue,ROAS
0,2020,2020-07,2020-07-27 - 2020-08-02,7/31/2020,unknown,Snapchat,ios,Expired Attributions,38498,0.0077,...,5026318,0,0,0,38498,0,0,0,0.0,0.0
1,2020,2020-08,2020-07-27 - 2020-08-02,8/2/2020,unknown,Snapchat,ios,Expired Attributions,37438,0.0080,...,4695524,0,0,0,37438,0,0,0,0.0,0.0
2,2020,2020-08,2020-08-03 - 2020-08-09,8/7/2020,unknown,Snapchat,ios,Expired Attributions,32776,0.0077,...,4264904,0,0,0,32776,0,0,0,0.0,0.0
3,2020,2020-07,2020-07-06 - 2020-07-12,7/7/2020,unknown,Snapchat,ios,Expired Attributions,31644,0.0081,...,3930864,0,0,0,31644,0,0,0,0.0,0.0
4,2020,2020-07,2020-07-06 - 2020-07-12,7/8/2020,unknown,Snapchat,ios,Expired Attributions,30749,0.0081,...,3797587,0,0,0,30749,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151020,2022,2022-07,2022-06-27 - 2022-07-03,7/3/2022,Singapore,Organic,ios,unknown,0,0.0000,...,0,0,0,0,0,1,0,0,0.0,0.0
151021,2022,2022-07,2022-06-27 - 2022-07-03,7/3/2022,Thailand,Organic,ios,unknown,0,0.0000,...,0,0,0,0,0,1,0,0,0.0,0.0
151022,2022,2022-07,2022-06-27 - 2022-07-03,7/3/2022,Turkey,Organic,ios,unknown,0,0.0000,...,0,0,0,0,0,8,0,0,0.0,0.0
151023,2022,2022-07,2022-06-27 - 2022-07-03,7/3/2022,United States,Organic,ios,unknown,0,0.0000,...,0,0,0,0,0,11,0,0,0.0,0.0


In [4]:
df=df.drop(['Week'],axis='columns')

In [5]:
df['Month']=df['Month'].astype(str)
df['Month']=df['Month'].str.split("-")
df['Month']=df['Month'].apply(lambda x:x[1]).astype(int)

df['DayNumber']=df['Day'].str.split("/")
df['DayNumber']=df['DayNumber'].apply(lambda x:x[1]).astype(int)

# df=df.rename(columns = {'Campaign':'Campaign Name'})

In [6]:
df['Day']=pd.to_datetime(df['Day'])

In [7]:
df=df.replace(float('inf'),0.0)
df=df.fillna(0)

df=df.sort_values(['Day'])

In [8]:
df.columns

Index(['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'Revenue', 'ROAS', 'DayNumber'],
      dtype='object')

In [9]:
rbuild=df.copy()


In [10]:
last_date=rbuild.iloc[rbuild.shape[0]-1,2]
last_date=pd.to_datetime(last_date)

pivot_date=last_date-timedelta(days=days_to_fix)
pivot_date=pd.to_datetime(pivot_date)

first_day=rbuild.iloc[0,2]
first_day=pd.to_datetime(first_day)

In [11]:
first_day

Timestamp('2020-04-01 00:00:00')

In [12]:
mask=(rbuild['Day']>=first_day) & (rbuild['Day']<pivot_date)
rbuild=rbuild.loc[mask]

In [13]:
rbuild['Revenue'].unique()

array([0.00000e+00, 3.98600e+01, 1.56593e+02, ..., 9.65000e-02,
       3.20000e+00, 1.22600e+00])

In [14]:
Y=rbuild['Revenue']
X=rbuild.drop(['Revenue','ROAS'],axis='columns')

In [15]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['Campaign']=le.fit_transform(X['Campaign'])
X['OS name']=le.fit_transform(X['OS name'])
X['Country']=le.fit_transform(X['Country'])
X['Channel']=le.fit_transform(X['Channel'])

In [16]:
X=X.drop(['Day'],axis='columns')

In [17]:
X.columns

Index(['Year', 'Month', 'Country', 'Channel', 'OS name', 'Campaign', 'Clicks',
       'CTR', 'Impressions', 'Installs', 'Ad Spend', 'Installs (paid)',
       'Impressions (paid)', 'Impressions (Network)', 'Installs (Network)',
       'Clicks (Network)', 'Clicks (paid)', 'Organic Installs',
       'Non-Organic Installs', 'Total Installs (SKAdNetwork)', 'DayNumber'],
      dtype='object')

In [18]:
from xgboost import XGBRegressor
xgb=XGBRegressor(objective='count:poisson',n_estimators=250)
xgb.fit(X,Y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0.699999988, max_depth=6, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=250, n_jobs=0, num_parallel_tree=1,
             objective='count:poisson', predictor='auto', random_state=0,
             reg_alpha=0, ...)

In [19]:
rtest=df.copy()

In [20]:
rtest

,Year,Month,Day,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,...,Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),Revenue,ROAS,DayNumber
16965,2020,4,2020-04-01,Egypt,TikTok for Business,ios,VIP Baloot&1657526815513637,2,0.0294,68,...,0,0,0,0,0,1,0,0.00,0.0,1
105228,2020,4,2020-04-01,Kuwait,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,33,0,0,39.86,0.0,1
105227,2020,4,2020-04-01,Jordan,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,38,0,0,0.00,0.0,1
105226,2020,4,2020-04-01,Iraq,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,16,0,0,0.00,0.0,1
105225,2020,4,2020-04-01,India,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,3,0,0,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21439,2022,7,2022-07-03,Kuwait,Snapchat,ios,BAL_SC_UA_I_SKAD_SA-KW_MAI (1ecb9265-7bb8-4f39...,0,0.0000,0,...,341531,5,112,0,0,0,0,0.00,0.0,3
21440,2022,7,2022-07-03,Saudi Arabia,Snapchat,ios,BAL_SC_UA_I_SKAD_SA-KW_MAI (1ecb9265-7bb8-4f39...,0,0.0000,0,...,1384265,71,1911,0,0,18,0,0.00,0.0,3
21441,2022,7,2022-07-03,unknown,Snapchat,ios,VIP Baloot_I_SA_SKAd_AEO (41130874-6e52-490e-b...,0,0.0000,0,...,0,4,0,0,0,0,3,0.00,0.0,3
35791,2022,7,2022-07-03,Egypt,TikTok for Business,ios,BAL_TT_RT_I_SKAD_WW (1729075105810449),0,0.0000,0,...,0,1,0,0,0,0,0,0.00,0.0,3


In [21]:
mask=(rtest['Day']<=last_date) & (rtest['Day']>=pivot_date)
rtest=rtest.loc[mask]

In [22]:
rtest

,Year,Month,Day,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,...,Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),Revenue,ROAS,DayNumber
147690,2022,5,2022-05-04,United Arab Emirates,Organic,ios,unknown,0,0.000,0,...,0,0,0,0,47,0,0,173.5698,0.0,4
147697,2022,5,2022-05-04,Egypt,Organic,ios,unknown,0,0.000,0,...,0,0,0,0,8,0,0,0.0532,0.0,4
147689,2022,5,2022-05-04,Yemen,Organic,android,unknown,0,0.000,0,...,0,0,0,0,19,0,0,0.0272,0.0,4
12400,2022,5,2022-05-04,United States,Unity Ads,ios,BAL_Unity_UA_I_SKAD_SA_MAI_WL (6257f26cd0a9eb4...,22,0.057,386,...,0,0,0,0,0,0,51,0.0000,0.0,4
68357,2022,5,2022-05-04,Iraq,Apple,ios,BAL_ASA_UA_I_US_CPP (1036672884),0,0.000,0,...,0,0,0,0,0,1,0,0.0000,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21439,2022,7,2022-07-03,Kuwait,Snapchat,ios,BAL_SC_UA_I_SKAD_SA-KW_MAI (1ecb9265-7bb8-4f39...,0,0.000,0,...,341531,5,112,0,0,0,0,0.0000,0.0,3
21440,2022,7,2022-07-03,Saudi Arabia,Snapchat,ios,BAL_SC_UA_I_SKAD_SA-KW_MAI (1ecb9265-7bb8-4f39...,0,0.000,0,...,1384265,71,1911,0,0,18,0,0.0000,0.0,3
21441,2022,7,2022-07-03,unknown,Snapchat,ios,VIP Baloot_I_SA_SKAd_AEO (41130874-6e52-490e-b...,0,0.000,0,...,0,4,0,0,0,0,3,0.0000,0.0,3
35791,2022,7,2022-07-03,Egypt,TikTok for Business,ios,BAL_TT_RT_I_SKAD_WW (1729075105810449),0,0.000,0,...,0,1,0,0,0,0,0,0.0000,0.0,3


In [23]:
Y=rtest['Revenue']
X=rtest.drop(['Revenue','ROAS'],axis='columns')

In [24]:
X.columns

Index(['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'DayNumber'],
      dtype='object')

In [25]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['Campaign']=le.fit_transform(X['Campaign'])
X['OS name']=le.fit_transform(X['OS name'])
X['Country']=le.fit_transform(X['Country'])
X['Channel']=le.fit_transform(X['Channel'])

In [26]:
X=X.drop(['Day'],axis='columns')

In [27]:
df2=df.copy()

In [28]:
thirty_revenue=xgb.predict(X)

In [29]:
thirty_revenue

array([1.8088756 , 0.26542258, 0.4136209 , ..., 0.20084673, 0.15542625,
       0.32382613], dtype=float32)

In [30]:
rtest['Revenue']=thirty_revenue
rtest['ROAS']=thirty_revenue/rtest['Ad Spend']

In [31]:
rtest=rtest.replace(float('inf'),0.0)

In [32]:
rtest.to_csv('thirty_days_revenue.csv')

In [33]:
build_final=pd.concat([rbuild,rtest])

In [34]:
build_final.columns

Index(['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'Revenue', 'ROAS', 'DayNumber'],
      dtype='object')

In [35]:
ind=build_final[build_final['Day']==last_date].index
build_final=build_final.drop(ind)

In [36]:
build_final.to_csv('final_data.csv')

In [37]:
build_final

,Year,Month,Day,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,...,Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),Revenue,ROAS,DayNumber
16965,2020,4,2020-04-01,Egypt,TikTok for Business,ios,VIP Baloot&1657526815513637,2,0.0294,68,...,0,0,0,0,0,1,0,0.000000,0.000000,1
105228,2020,4,2020-04-01,Kuwait,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,33,0,0,39.860000,0.000000,1
105227,2020,4,2020-04-01,Jordan,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,38,0,0,0.000000,0.000000,1
105226,2020,4,2020-04-01,Iraq,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,16,0,0,0.000000,0.000000,1
105225,2020,4,2020-04-01,India,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,3,0,0,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9593,2022,7,2022-07-02,unknown,Apple,ios,BAL_ASA_UA_I_SA_CPA-TEST2 (1075894419),51,0.1678,304,...,0,0,0,51,0,0,0,0.029895,0.000000,2
3107,2022,7,2022-07-02,Saudi Arabia,Unity Ads,ios,BAL_Unity_UA_I_SKAD_SA_MAI_WL (6257f26cd0a9eb4...,585,0.0719,8138,...,9293,16,571,0,0,16,10,0.017637,0.000177,2
9285,2022,7,2022-07-02,Kuwait,TikTok for Business,server,BAL_TT_UA_I_SKAD_SA-KW_MAI (1729998279267345),56,0.0078,7139,...,0,0,0,0,0,0,0,0.008085,0.000000,2
15066,2022,7,2022-07-02,Germany,TikTok for Business,android,BAL_TT_RT_A_WW (1729074737227825),6,0.0187,321,...,0,0,0,0,0,0,0,0.011148,0.000000,2


In [38]:
predict=build_final.copy()

In [39]:
last_date=predict.iloc[predict.shape[0]-1,2]
last_date=pd.to_datetime(last_date)
last_date=last_date-timedelta(days=1)
# last_date_index=df2.shape[0]-1
lastdatecheck=predict[predict['Day']==last_date]
pre_active_campaign=lastdatecheck['Campaign'].unique()

In [40]:
predict=predict.replace(float('inf'),0.0)
predict=predict.fillna(0.0)

In [41]:
predict

,Year,Month,Day,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,...,Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),Revenue,ROAS,DayNumber
16965,2020,4,2020-04-01,Egypt,TikTok for Business,ios,VIP Baloot&1657526815513637,2,0.0294,68,...,0,0,0,0,0,1,0,0.000000,0.000000,1
105228,2020,4,2020-04-01,Kuwait,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,33,0,0,39.860000,0.000000,1
105227,2020,4,2020-04-01,Jordan,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,38,0,0,0.000000,0.000000,1
105226,2020,4,2020-04-01,Iraq,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,16,0,0,0.000000,0.000000,1
105225,2020,4,2020-04-01,India,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,3,0,0,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9593,2022,7,2022-07-02,unknown,Apple,ios,BAL_ASA_UA_I_SA_CPA-TEST2 (1075894419),51,0.1678,304,...,0,0,0,51,0,0,0,0.029895,0.000000,2
3107,2022,7,2022-07-02,Saudi Arabia,Unity Ads,ios,BAL_Unity_UA_I_SKAD_SA_MAI_WL (6257f26cd0a9eb4...,585,0.0719,8138,...,9293,16,571,0,0,16,10,0.017637,0.000177,2
9285,2022,7,2022-07-02,Kuwait,TikTok for Business,server,BAL_TT_UA_I_SKAD_SA-KW_MAI (1729998279267345),56,0.0078,7139,...,0,0,0,0,0,0,0,0.008085,0.000000,2
15066,2022,7,2022-07-02,Germany,TikTok for Business,android,BAL_TT_RT_A_WW (1729074737227825),6,0.0187,321,...,0,0,0,0,0,0,0,0.011148,0.000000,2


In [42]:
predict.columns

Index(['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'Revenue', 'ROAS', 'DayNumber'],
      dtype='object')

In [43]:
def campaign_ads(predict,ads):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        ads_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]

        add_date=pivot_date
        add_date=pd.to_datetime(add_date)
        for i in range(prediction_ma_window):       
            sub2=sub.loc[sub['Day']==add_date]
            ads_minor.append(np.sum(sub2['Ad Spend']))
            add_date=add_date+timedelta(days=1)
            add_date=pd.to_datetime(add_date) 
        start=0
        end=ads_ma

        while(end<ads_ma+days_to_predict):
            ma=np.mean(ads_minor[start:end+1])
            ads_minor.append(ma)
            start+=1
            end+=1

        ads.append(ads_minor)   

In [44]:
def campaign_ctr(predict,ctr):
    for i in range(len(pre_active_campaign)):
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        ctr_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        ctr_minor.append(sub['CTR'].min())
        ctr_minor.append(sub['CTR'].max())
        ctr_minor.append(sub['CTR'].median())
        
        ctr.append(ctr_minor)
 

In [45]:
def campaign_impressions(predict,impressions):
    for i in range(len(pre_active_campaign)):
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        impressions_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        impressions_minor.append(sub['Impressions'].min())
        impressions_minor.append(sub['Impressions'].max())
        impressions_minor.append(sub['Impressions'].median())
        
        impressions.append(impressions_minor)
 

In [46]:
def campaign_impressions2(predict,impressions2):
    for i in range(len(pre_active_campaign)):
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        impressions_minor2=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        impressions_minor2.append(sub['Impressions (paid)'].min())
        impressions_minor2.append(sub['Impressions (paid)'].max())
        impressions_minor2.append(sub['Impressions (paid)'].median())
        
        impressions2.append(impressions_minor2)
 

In [47]:
def campaign_impressions3(predict,impressions3):
    for i in range(len(pre_active_campaign)):
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        impressions_minor3=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        impressions_minor3.append(sub['Impressions (Network)'].min())
        impressions_minor3.append(sub['Impressions (Network)'].max())
        impressions_minor3.append(sub['Impressions (Network)'].median())
        
        impressions3.append(impressions_minor3)
 

In [48]:
def campaign_installs(predict,installs):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        installs_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        value=sub['Installs'].min()
        installs_minor.append(value)
        installs_minor.append(sub['Installs'].max())
        installs_minor.append(sub['Installs'].median())
        
        installs.append(installs_minor)


In [49]:
def campaign_installsp(predict,installsp):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        installsp_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        installsp_minor.append(sub['Installs (paid)'].min())
        installsp_minor.append(sub['Installs (paid)'].max())
        installsp_minor.append(sub['Installs (paid)'].median())
        
        installsp.append(installsp_minor)
          

In [50]:
def campaign_installs2(predict,installs2):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        installs2_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        installs2_minor.append(sub['Installs (Network)'].min())
        installs2_minor.append(sub['Installs (Network)'].max())
        installs2_minor.append(sub['Installs (Network)'].median())
        
        installs2.append(installs2_minor)
          

In [51]:
def campaign_installs3(predict,installs3):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        installs3_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        installs3_minor.append(sub['Total Installs (SKAdNetwork)'].min())
        installs3_minor.append(sub['Total Installs (SKAdNetwork)'].max())
        installs3_minor.append(sub['Total Installs (SKAdNetwork)'].median())
        
        installs3.append(installs3_minor)
          

In [52]:
def campaign_organic(predict,organic):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        organic_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        organic_minor.append(sub['Organic Installs'].min())
        organic_minor.append(sub['Organic Installs'].max())
        organic_minor.append(sub['Organic Installs'].median())
        
        organic.append(organic_minor)
 

In [53]:
def campaign_organic2(predict,organic2):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        organic2_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        organic2_minor.append(sub['Non-Organic Installs'].min())
        organic2_minor.append(sub['Non-Organic Installs'].max())
        organic2_minor.append(sub['Non-Organic Installs'].median())
        
        organic2.append(organic2_minor)
 

In [54]:
def campaign_clicks(predict,clicks):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        clicks_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        clicks_minor.append(sub['Clicks'].min())
        clicks_minor.append(sub['Clicks'].max())
        clicks_minor.append(sub['Clicks'].median())
        
        clicks.append(clicks_minor)
 

In [55]:
def campaign_clicks2(predict,clicks2):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        clicks2_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        clicks2_minor.append(sub['Clicks (Network)'].min())
        clicks2_minor.append(sub['Clicks (Network)'].max())
        clicks2_minor.append(sub['Clicks (Network)'].median())
        
        clicks2.append(clicks2_minor)
 

In [56]:
def campaign_clicks3(predict,clicks3):
    for i in range(len(pre_active_campaign)):
        print(i)
        df3=predict[predict['Campaign']==pre_active_campaign[i]]

        last_date=df3.iloc[df3.shape[0]-1,2]
        last_date=pd.to_datetime(last_date)

        pivot_date=last_date-timedelta(days=ads_ma)
        pivot_date=pd.to_datetime(pivot_date)

        df3=df3.replace(float('inf'),0.0)
        df3=df3.fillna(0)

        clicks3_minor=[]
        mask=(df3['Day']>=pivot_date) & (df3['Day']<=last_date)
        sub=df3.loc[mask]
        
        clicks3_minor.append(sub['Clicks (paid)'].min())
        clicks3_minor.append(sub['Clicks (paid)'].max())
        clicks3_minor.append(sub['Clicks (paid)'].median())
        
        clicks3.append(clicks3_minor)
 

In [57]:
ads=[]
ctr=[]
impressions=[]
impressions2=[]
impressions3=[]
installs=[]
installsp=[]
installs2=[]
installs3=[]
organic=[]
organic2=[]
clicks=[]
clicks2=[]
clicks3=[]

campaign_ads(predict,ads)
campaign_ctr(predict,ctr)
campaign_impressions(predict,impressions)
campaign_impressions2(predict,impressions2)
campaign_impressions3(predict,impressions3)
campaign_installs(predict,installs)
campaign_installsp(predict,installsp)
campaign_installs2(predict,installs2)
campaign_installs3(predict,installs3)
campaign_organic(predict,organic)
campaign_organic2(predict,organic2)
campaign_clicks(predict,clicks)
campaign_clicks2(predict,clicks2)
campaign_clicks3(predict,clicks3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [58]:
clicks[5]

[0, 1950, 44.5]

In [59]:
final=pd.DataFrame()

In [60]:
predict

,Year,Month,Day,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,...,Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),Revenue,ROAS,DayNumber
16965,2020,4,2020-04-01,Egypt,TikTok for Business,ios,VIP Baloot&1657526815513637,2,0.0294,68,...,0,0,0,0,0,1,0,0.000000,0.000000,1
105228,2020,4,2020-04-01,Kuwait,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,33,0,0,39.860000,0.000000,1
105227,2020,4,2020-04-01,Jordan,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,38,0,0,0.000000,0.000000,1
105226,2020,4,2020-04-01,Iraq,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,16,0,0,0.000000,0.000000,1
105225,2020,4,2020-04-01,India,Organic,android,unknown,0,0.0000,0,...,0,0,0,0,3,0,0,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9593,2022,7,2022-07-02,unknown,Apple,ios,BAL_ASA_UA_I_SA_CPA-TEST2 (1075894419),51,0.1678,304,...,0,0,0,51,0,0,0,0.029895,0.000000,2
3107,2022,7,2022-07-02,Saudi Arabia,Unity Ads,ios,BAL_Unity_UA_I_SKAD_SA_MAI_WL (6257f26cd0a9eb4...,585,0.0719,8138,...,9293,16,571,0,0,16,10,0.017637,0.000177,2
9285,2022,7,2022-07-02,Kuwait,TikTok for Business,server,BAL_TT_UA_I_SKAD_SA-KW_MAI (1729998279267345),56,0.0078,7139,...,0,0,0,0,0,0,0,0.008085,0.000000,2
15066,2022,7,2022-07-02,Germany,TikTok for Business,android,BAL_TT_RT_A_WW (1729074737227825),6,0.0187,321,...,0,0,0,0,0,0,0,0.011148,0.000000,2


In [61]:
last_date=predict.iloc[predict.shape[0]-1,2]
last_date=pd.to_datetime(last_date)
for j in range(days_to_predict):
    print(j)
    test2=pd.DataFrame()
    last_date=last_date+timedelta(days=1)

#     test2['Year']=last_date.year
#     test2['Month']=last_date.month

    predict=predict.replace(float('inf'),0.0)
    predict=predict.fillna(0)

    nullcc=predict[predict['Country']==0].index
    predict=predict.drop(nullcc)

    cc=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['Country'].unique()[0]
        cc.append(sample)

    os=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['OS name'].unique()[0]
        os.append(sample)


    channel=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['Channel'].unique()[0]
        channel.append(sample)

    test2['Country']=cc
    test2['Channel']=channel
    test2['OS name']=os
    test2['Campaign']=pre_active_campaign
#     c+=1

    ads_2=[]
    ctr_2=[]
    impressions_2=[]
    impressions2_2=[]
    impressions3_2=[]
    installs_2=[]
    installsp_2=[]
    installs2_2=[]
    installs3_2=[]
    organic_2=[]
    organic2_2=[]
    clicks_2=[]
    clicks2_2=[]
    clicks3_2=[]

    for k in range(len(pre_active_campaign)):
#         len(ads[j])-days_to_predict-1+i
        ads_2.append(ads[k][len(ads[k])-days_to_predict-1+i])
        ctr_2.append(ctr[k][0])
        impressions_2.append(impressions[k][0])
        impressions2_2.append(impressions2[k][0])
        impressions3_2.append(impressions3[k][0])
        installs_2.append(installs[k][0])
        installsp_2.append(installsp[k][0])
        installs2_2.append(installs2[k][0])
        installs3_2.append(installs3[k][0])
        organic_2.append(organic[k][0])
        organic2_2.append(organic2[k][0])
        clicks_2.append(clicks[k][0])
        clicks2_2.append(clicks2[k][0])
        clicks3_2.append(clicks3[k][0])
        
#     ['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
#        'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
#        'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
#        'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
#        'Organic Installs', 'Non-Organic Installs',
#        'Total Installs (SKAdNetwork)', 'Revenue', 'ROAS', 'DayNumber']

    test2['Ad Spend']=ads_2
    test2['CTR']=ctr_2
    test2['Impressions']=impressions_2
    test2['Impressions (paid)']=impressions2_2
    test2['Impressions (Network)']=impressions3_2
    test2['Installs']=installs_2
    test2['Installs (paid)']=installsp_2
    test2['Installs (Network)']=installs2_2
    test2['Total Installs (SKAdNetwork)']=installs3_2
    test2['Clicks']=clicks_2
    test2['Clicks (Network)']=clicks2_2
    test2['Clicks (paid)']=clicks3_2
    test2['Organic Installs']=organic_2
    test2['Non-Organic Installs']=organic2_2
    
    test2['DayNumber']=last_date.day

    test2['Year']=last_date.year
    test2['Month']=last_date.month

    test2=test2[['Year', 'Month', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'DayNumber']]

    final=pd.concat([test2,final])

    final=final.reset_index(drop=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [62]:
final

,Year,Month,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,Installs,...,Installs (paid),Impressions (paid),Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),DayNumber
0,2022,8,United States,TikTok for Business,android,BAL_TT_RT_A_WW (1729074737227825),0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2022,8,Saudi Arabia,Unity Ads,macos,BAL_Unity_UA_I_SKAD_SA_MAI_WL (6257f26cd0a9eb4...,0,0.0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,2022,8,Kuwait,Organic,android,unknown,0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2022,8,United States,Unity Ads,ios,BAL_Unity_UA_I_SKAD_SA_ROAS (627ceb9a03bd83972...,0,0.0,1,0,...,0,1,0,0,0,0,0,0,0,1
4,2022,8,Norway,TikTok for Business,server,BAL_TT_UA_I_SKAD_SA-KW_MAI (1729998279267345),0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2022,7,United Arab Emirates,TikTok for Business,ios,BAL_TT_RT_I_SKAD_WW (1729075105810449),0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,3
716,2022,7,Kuwait,TikTok for Business,android,VIP Baloot_A_SA_KW_AEO (1726285843476530),0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,3
717,2022,7,unknown,Snapchat,ios,VIP Baloot_I_SA_iOS14_Purchase (65c67d08-416f-...,0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,3
718,2022,7,Saudi Arabia,Snapchat,ios,VIP Baloot_I_SA_SKAd_AEO (41130874-6e52-490e-b...,0,0.0,0,0,...,0,0,0,4,0,0,0,0,1,3


In [63]:
pfinal=final.copy()
pfinal_2=final.copy()
final_2=final.copy()
pfinal_3=final.copy()
final_3=final.copy()

In [64]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
pfinal['Campaign']=le.fit_transform(pfinal['Campaign'])
pfinal['OS name']=le.fit_transform(pfinal['OS name'])
pfinal['Country']=le.fit_transform(pfinal['Country'])
pfinal['Channel']=le.fit_transform(pfinal['Channel'])


In [65]:
min_revenue=xgb.predict(pfinal)

In [66]:
final['Min Revenue']=min_revenue
final['ROAS']=final['Min Revenue']/final['Ad Spend']

In [67]:
final.to_csv('Min_Revenue.csv')

In [68]:
final['Ad Spend'].unique()

array([ 100.        ,   99.92575024,    0.        ,  199.92305136,
       1083.33338619,  100.68499221,    4.95999169,   10.18665503,
         16.44440289,  252.32834049,   41.96665092,   59.08999779,
        352.8766752 ,  147.46336247,   25.        ])

In [69]:
final_2['Ad Spend']=1.5*final_2['Ad Spend']
pfinal_2['Ad Spend']=1.5*pfinal_2['Ad Spend']
pfinal_2['Campaign']=le.fit_transform(pfinal_2['Campaign'])
pfinal_2['OS name']=le.fit_transform(pfinal_2['OS name'])
pfinal_2['Country']=le.fit_transform(pfinal_2['Country'])
pfinal_2['Channel']=le.fit_transform(pfinal_2['Channel'])
min_revenue=xgb.predict(pfinal_2)
final_2['Min Revenue']=min_revenue
final_2['ROAS']=final_2['Min Revenue']/final_2['Ad Spend']
final_2.to_csv('Min_Revenue_1.5.csv')

In [70]:
final_2['Ad Spend'].unique()

array([ 150.        ,  149.88862536,    0.        ,  299.88457704,
       1625.00007929,  151.02748831,    7.43998753,   15.27998254,
         24.66660433,  378.49251074,   62.94997638,   88.63499668,
        529.3150128 ,  221.19504371,   37.5       ])

In [71]:
final_3['Ad Spend']=2*final_3['Ad Spend']
pfinal_3['Ad Spend']=2*pfinal_3['Ad Spend']
pfinal_3['Campaign']=le.fit_transform(pfinal_3['Campaign'])
pfinal_3['OS name']=le.fit_transform(pfinal_3['OS name'])
pfinal_3['Country']=le.fit_transform(pfinal_3['Country'])
pfinal_3['Channel']=le.fit_transform(pfinal_3['Channel'])
min_revenue=xgb.predict(pfinal_3)
final_3['Min Revenue']=min_revenue
final_3['ROAS']=final_3['Min Revenue']/final_3['Ad Spend']
final_3.to_csv('Min_Revenue_2.csv')

In [72]:
final_3['Ad Spend'].unique()

array([ 200.        ,  199.85150047,    0.        ,  399.84610272,
       2166.66677239,  201.36998442,    9.91998338,   20.37331006,
         32.88880577,  504.65668098,   83.93330185,  118.17999557,
        705.7533504 ,  294.92672494,   50.        ])

In [73]:
final2=pd.DataFrame()
last_date=predict.iloc[predict.shape[0]-1,2]
last_date=pd.to_datetime(last_date)
for j in range(days_to_predict):
    print(j)
    test2=pd.DataFrame()
    last_date=last_date+timedelta(days=1)

#     test2['Year']=last_date.year
#     test2['Month']=last_date.month

    predict=predict.replace(float('inf'),0.0)
    predict=predict.fillna(0)

    nullcc=predict[predict['Country']==0].index
    predict=predict.drop(nullcc)

    cc=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['Country'].unique()[0]
        cc.append(sample)

    os=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['OS name'].unique()[0]
        os.append(sample)


    channel=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['Channel'].unique()[0]
        channel.append(sample)

    test2['Country']=cc
    test2['Channel']=channel
    test2['OS name']=os
    test2['Campaign']=pre_active_campaign
#     c+=1

    ads_2=[]
    ctr_2=[]
    impressions_2=[]
    impressions2_2=[]
    impressions3_2=[]
    installs_2=[]
    installsp_2=[]
    installs2_2=[]
    installs3_2=[]
    organic_2=[]
    organic2_2=[]
    clicks_2=[]
    clicks2_2=[]
    clicks3_2=[]

    for k in range(len(pre_active_campaign)):
#         len(ads[j])-days_to_predict-1+i
        ads_2.append(ads[k][len(ads[k])-days_to_predict-1+i])
        ctr_2.append(ctr[k][1])
        impressions_2.append(impressions[k][1])
        impressions2_2.append(impressions2[k][1])
        impressions3_2.append(impressions3[k][1])
        installs_2.append(installs[k][1])
        installsp_2.append(installsp[k][1])
        installs2_2.append(installs2[k][1])
        installs3_2.append(installs3[k][1])
        organic_2.append(organic[k][1])
        organic2_2.append(organic2[k][1])
        clicks_2.append(clicks[k][1])
        clicks2_2.append(clicks2[k][1])
        clicks3_2.append(clicks3[k][1])
        
#     ['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
#        'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
#        'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
#        'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
#        'Organic Installs', 'Non-Organic Installs',
#        'Total Installs (SKAdNetwork)', 'Revenue', 'ROAS', 'DayNumber']

    test2['Ad Spend']=ads_2
    test2['CTR']=ctr_2
    test2['Impressions']=impressions_2
    test2['Impressions (paid)']=impressions2_2
    test2['Impressions (Network)']=impressions3_2
    test2['Installs']=installs_2
    test2['Installs (paid)']=installsp_2
    test2['Installs (Network)']=installs2_2
    test2['Total Installs (SKAdNetwork)']=installs3_2
    test2['Clicks']=clicks_2
    test2['Clicks (Network)']=clicks2_2
    test2['Clicks (paid)']=clicks3_2
    test2['Organic Installs']=organic_2
    test2['Non-Organic Installs']=organic2_2
    
    test2['DayNumber']=last_date.day

    test2['Year']=last_date.year
    test2['Month']=last_date.month

    test2=test2[['Year', 'Month', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'DayNumber']]

    final2=pd.concat([test2,final2])

    final2=final2.reset_index(drop=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [74]:
final2

,Year,Month,Country,Channel,OS name,Campaign,Clicks,CTR,Impressions,Installs,...,Installs (paid),Impressions (paid),Impressions (Network),Installs (Network),Clicks (Network),Clicks (paid),Organic Installs,Non-Organic Installs,Total Installs (SKAdNetwork),DayNumber
0,2022,8,United States,TikTok for Business,android,BAL_TT_RT_A_WW (1729074737227825),2148,909.0,77843,11,...,0,0,60759,150,3937,0,0,11,0,1
1,2022,8,Saudi Arabia,Unity Ads,macos,BAL_Unity_UA_I_SKAD_SA_MAI_WL (6257f26cd0a9eb4...,585,1.0,9719,18,...,0,9719,10832,18,616,0,0,18,51,1
2,2022,8,Kuwait,Organic,android,unknown,0,0.0,3,1503,...,0,0,0,0,0,0,1503,0,0,1
3,2022,8,United States,Unity Ads,ios,BAL_Unity_UA_I_SKAD_SA_ROAS (627ceb9a03bd83972...,1039,1.0,19765,38,...,0,19765,21998,38,1018,0,0,38,50,1
4,2022,8,Norway,TikTok for Business,server,BAL_TT_UA_I_SKAD_SA-KW_MAI (1729998279267345),393,1.0,43331,9,...,0,0,80081,56,646,0,0,9,56,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2022,7,United Arab Emirates,TikTok for Business,ios,BAL_TT_RT_I_SKAD_WW (1729075105810449),0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,3
716,2022,7,Kuwait,TikTok for Business,android,VIP Baloot_A_SA_KW_AEO (1726285843476530),0,0.0,10,0,...,0,0,0,0,0,0,0,0,0,3
717,2022,7,unknown,Snapchat,ios,VIP Baloot_I_SA_iOS14_Purchase (65c67d08-416f-...,0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,3
718,2022,7,Saudi Arabia,Snapchat,ios,VIP Baloot_I_SA_SKAd_AEO (41130874-6e52-490e-b...,0,0.0,0,0,...,0,0,0,18,0,0,0,0,6,3


In [75]:

pfinal2=final2.copy()
pfinal2_2=final2.copy()
final2_2=final2.copy()
pfinal2_3=final2.copy()
final2_3=final2.copy()

In [76]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
pfinal2['Campaign']=le.fit_transform(pfinal2['Campaign'])
pfinal2['OS name']=le.fit_transform(pfinal2['OS name'])
pfinal2['Country']=le.fit_transform(pfinal2['Country'])
pfinal2['Channel']=le.fit_transform(pfinal2['Channel'])

In [77]:
max_revenue=xgb.predict(pfinal2)

In [78]:
final2['Max Revenue']=max_revenue
final2['ROAS']=final2['Max Revenue']/final2['Ad Spend']

In [79]:
final2.to_csv('Max_Revenue.csv')

In [80]:
final2_2['Ad Spend']=1.5*final2_2['Ad Spend']
pfinal2_2['Ad Spend']=1.5*pfinal2_2['Ad Spend']
pfinal2_2['Campaign']=le.fit_transform(pfinal2_2['Campaign'])
pfinal2_2['OS name']=le.fit_transform(pfinal2_2['OS name'])
pfinal2_2['Country']=le.fit_transform(pfinal2_2['Country'])
pfinal2_2['Channel']=le.fit_transform(pfinal2_2['Channel'])
max_revenue=xgb.predict(pfinal2_2)
final2_2['Max Revenue']=max_revenue
final2_2['ROAS']=final2_2['Max Revenue']/final2_2['Ad Spend']
final2_2.to_csv('Max_Revenue_1.5.csv')

In [81]:
final2_3['Ad Spend']=2*final2_3['Ad Spend']
pfinal2_3['Ad Spend']=2*pfinal2_3['Ad Spend']
pfinal2_3['Campaign']=le.fit_transform(pfinal2_3['Campaign'])
pfinal2_3['OS name']=le.fit_transform(pfinal2_3['OS name'])
pfinal2_3['Country']=le.fit_transform(pfinal2_3['Country'])
pfinal2_3['Channel']=le.fit_transform(pfinal2_3['Channel'])
max_revenue=xgb.predict(pfinal2_3)
final2_3['Max Revenue']=max_revenue
final2_3['ROAS']=final2_3['Max Revenue']/final2_3['Ad Spend']
final2_3.to_csv('Max_Revenue_2.csv')

In [82]:
final3=pd.DataFrame()
last_date=predict.iloc[predict.shape[0]-1,2]
last_date=pd.to_datetime(last_date)
for j in range(days_to_predict):
    print(j)
    test2=pd.DataFrame()
    last_date=last_date+timedelta(days=1)

#     test2['Year']=last_date.year
#     test2['Month']=last_date.month

    predict=predict.replace(float('inf'),0.0)
    predict=predict.fillna(0)

    nullcc=predict[predict['Country']==0].index
    predict=predict.drop(nullcc)

    cc=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['Country'].unique()[0]
        cc.append(sample)

    os=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['OS name'].unique()[0]
        os.append(sample)


    channel=[]
    for i in range(len(pre_active_campaign)):
        sample=predict[predict['Campaign']==pre_active_campaign[i]]['Channel'].unique()[0]
        channel.append(sample)

    test2['Country']=cc
    test2['Channel']=channel
    test2['OS name']=os
    test2['Campaign']=pre_active_campaign
#     c+=1

    ads_2=[]
    ctr_2=[]
    impressions_2=[]
    impressions2_2=[]
    impressions3_2=[]
    installs_2=[]
    installsp_2=[]
    installs2_2=[]
    installs3_2=[]
    organic_2=[]
    organic2_2=[]
    clicks_2=[]
    clicks2_2=[]
    clicks3_2=[]

    for k in range(len(pre_active_campaign)):
#         len(ads[j])-days_to_predict-1+i
        ads_2.append(ads[k][len(ads[k])-days_to_predict-1+i])
        ctr_2.append(ctr[k][2])
        impressions_2.append(impressions[k][2])
        impressions2_2.append(impressions2[k][2])
        impressions3_2.append(impressions3[k][2])
        installs_2.append(installs[k][2])
        installsp_2.append(installsp[k][2])
        installs2_2.append(installs2[k][2])
        installs3_2.append(installs3[k][2])
        organic_2.append(organic[k][2])
        organic2_2.append(organic2[k][2])
        clicks_2.append(clicks[k][2])
        clicks2_2.append(clicks2[k][2])
        clicks3_2.append(clicks3[k][2])
        
#     ['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign',
#        'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
#        'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
#        'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
#        'Organic Installs', 'Non-Organic Installs',
#        'Total Installs (SKAdNetwork)', 'Revenue', 'ROAS', 'DayNumber']

    test2['Ad Spend']=ads_2
    test2['CTR']=ctr_2
    test2['Impressions']=impressions_2
    test2['Impressions (paid)']=impressions2_2
    test2['Impressions (Network)']=impressions3_2
    test2['Installs']=installs_2
    test2['Installs (paid)']=installsp_2
    test2['Installs (Network)']=installs2_2
    test2['Total Installs (SKAdNetwork)']=installs3_2
    test2['Clicks']=clicks_2
    test2['Clicks (Network)']=clicks2_2
    test2['Clicks (paid)']=clicks3_2
    test2['Organic Installs']=organic_2
    test2['Non-Organic Installs']=organic2_2
    
    test2['DayNumber']=last_date.day

    test2['Year']=last_date.year
    test2['Month']=last_date.month

    test2=test2[['Year', 'Month', 'Country', 'Channel', 'OS name', 'Campaign',
       'Clicks', 'CTR', 'Impressions', 'Installs', 'Ad Spend',
       'Installs (paid)', 'Impressions (paid)', 'Impressions (Network)',
       'Installs (Network)', 'Clicks (Network)', 'Clicks (paid)',
       'Organic Installs', 'Non-Organic Installs',
       'Total Installs (SKAdNetwork)', 'DayNumber']]

    final3=pd.concat([test2,final3])

    final3=final3.reset_index(drop=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [83]:
pfinal3=final3.copy()
pfinal3_2=final3.copy()
final3_2=final3.copy()
pfinal3_3=final3.copy()
final3_3=final3.copy()

In [84]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
pfinal3['Campaign']=le.fit_transform(pfinal3['Campaign'])
pfinal3['OS name']=le.fit_transform(pfinal3['OS name'])
pfinal3['Country']=le.fit_transform(pfinal3['Country'])
pfinal3['Channel']=le.fit_transform(pfinal3['Channel'])

In [85]:
median_revenue=xgb.predict(pfinal3)

In [86]:
final3['Median Revenue']=median_revenue
final3['ROAS']=final3['Median Revenue']/final3['Ad Spend']

In [87]:
final3.to_csv('Median_Revenue.csv')

In [88]:
final3_2['Ad Spend']=1.5*final3_2['Ad Spend']
pfinal3_2['Ad Spend']=1.5*pfinal3_2['Ad Spend']
pfinal3_2['Campaign']=le.fit_transform(pfinal3_2['Campaign'])
pfinal3_2['OS name']=le.fit_transform(pfinal3_2['OS name'])
pfinal3_2['Country']=le.fit_transform(pfinal3_2['Country'])
pfinal3_2['Channel']=le.fit_transform(pfinal3_2['Channel'])
median_revenue=xgb.predict(pfinal3_2)
final3_2['Median Revenue']=median_revenue
final3_2['ROAS']=final3_2['Median Revenue']/final3_2['Ad Spend']
final3_2.to_csv('Median_Revenue_1.5.csv')

In [89]:
final3_3['Ad Spend']=2*final3_3['Ad Spend']
pfinal3_3['Ad Spend']=2*pfinal3_3['Ad Spend']
pfinal3_3['Campaign']=le.fit_transform(pfinal3_3['Campaign'])
pfinal3_3['OS name']=le.fit_transform(pfinal3_3['OS name'])
pfinal3_3['Country']=le.fit_transform(pfinal3_3['Country'])
pfinal3_3['Channel']=le.fit_transform(pfinal3_3['Channel'])
median_revenue=xgb.predict(pfinal3_3)
final3_3['Median Revenue']=median_revenue
final3_3['ROAS']=final3_3['Median Revenue']/final3_3['Ad Spend']
final3_3.to_csv('Median_Revenue_2.csv')

In [90]:
# df2['AS/Impressions']=df2['Ad Spend']/df2['Impressions']
# df2['AS/Installs']=df2['Ad Spend']/df2['Installs']
# df2['AS/Installs(paid)']=df2['Ad Spend']/df2['Installs (paid)']
# df2['AS/Organic Installs']=df2['Ad Spend']/df2['Organic Installs']
# df2['AS/Clicks']=df2['Ad Spend']/df2['Clicks']
# df2['Quality Score']=df2['AS/Impressions']*df2['AS/Installs']*df2['AS/Installs(paid)']*df2['AS/Organic Installs']*df2['AS/Clicks']
# # df2['Quality Score']=1/df2['Quality Score']

In [91]:
# df2=df2.fillna(0)
# df2=df2.replace(float('inf'),0.0)

In [92]:
# df2['Day']=pd.to_datetime(df2['Day'])

In [93]:
# df2['Quality Score'].unique()

In [94]:
# df2['AS/Impressions']*df2['AS/Installs']

In [95]:
# last_date=df2.iloc[df2.shape[0]-1,2]
# last_date=pd.to_datetime(last_date)
# # last_date=last_date-timedelta(days=1)
# # last_date_index=df2.shape[0]-1
# lastdatecheck=df2[df2['Day']==last_date]
# pre_active_campaign=lastdatecheck['Campaign'].unique()
# # len(pre_active_campaign)
# # len(lastdatecheck['Campaign'].unique())

In [96]:
# pre_active_campaign = np. delete(pre_active_campaign,[3,12,19,23])

In [97]:
#3,12,19,23

In [98]:
# df2.columns

In [99]:
# df2=df2.sort_values(['Day'])

In [100]:
#3,12,19,23

In [101]:
#27 26 25 24 23 20

In [102]:
# def campaign_ads(df2,ads):
    
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])
        
#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)

#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)


#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)


#         ads_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):    
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             ads_minor.append(np.sum(sub2['Ad Spend']))

#         start=0
#         end=len(ads_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(ads_minor[start:end+1])
#             ads_minor.append(ma)
#             start+=1
#             end+=1
#         ads.append(ads_minor)

In [103]:
# def campaign_impressions(df2,impressions):
        
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])   #for debugging 


#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)

#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)

#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)

#         impressions_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):    
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             impressions_minor.append(np.mean(sub2['AS/Impressions']))

#         start=0
#         end=len(impressions_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(impressions_minor[start:end+1])
#             impressions_minor.append(ma)
#             start+=1
#             end+=1
#         impressions.append(impressions_minor)

In [104]:
# def campaign_installs(df2,installs):
        
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])   #for debugging 
        
#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)

#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)

#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)

#         installs_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):    
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             installs_minor.append(np.mean(sub2['AS/Installs']))

#         start=0
#         end=len(installs_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(installs_minor[start:end+1])
#             installs_minor.append(ma)
#             start+=1
#             end+=1
#         installs.append(installs_minor)

In [105]:
# def campaign_installsp(df2,installsp):
        
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])   #for debugging 
        
#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)


#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)
        
        
#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)

#         installsp_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):    
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             installsp_minor.append(np.mean(sub2['AS/Installs(paid)']))

#         start=0
#         end=len(installsp_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(installsp_minor[start:end+1])
#             installsp_minor.append(ma)
#             start+=1
#             end+=1
#         installsp.append(installsp_minor)

In [106]:
# def campaign_organic(df2,organic):
        
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])   #for debugging 
#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)


#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)
        
        
#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)

#         organic_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):    
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             organic_minor.append(np.mean(sub2['AS/Organic Installs']))

#         start=0
#         end=len(organic_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(organic_minor[start:end+1])
#             organic_minor.append(ma)
#             start+=1
#             end+=1
#         organic.append(organic_minor)

In [107]:
# def campaign_clicks(df2,clicks):
        
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])   #for debugging 
#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)

#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)
        
        
#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)

#         clicks_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):    
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             clicks_minor.append(np.mean(sub2['AS/Clicks']))

#         start=0
#         end=len(clicks_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(clicks_minor[start:end+1])
#             clicks_minor.append(ma)
#             start+=1
#             end+=1
#         clicks.append(clicks_minor)

In [108]:
# def campaign_revenue(df2,revenue):
        
#     for i in range(len(pre_active_campaign)):
#         print(pre_active_campaign[i])   #for debugging 
#         df3=df2[df2['Campaign']==pre_active_campaign[i]]
#         df3['Day']=pd.to_datetime(df3['Day'])
#         df3=df3.sort_values('Day')

#         last_date=df3.iloc[df3.shape[0]-1,2]
#         last_date=pd.to_datetime(last_date)


#         pivot_date=last_date-timedelta(days=30)
#         pivot_date=pd.to_datetime(pivot_date)
        
        
#         top_pivot_date=pivot_date-timedelta(days=ma_window)
#         top_pivot_date=pd.to_datetime(top_pivot_date)

#         df3=df3.replace(float('inf'),0.0)
#         df3=df3.fillna(0)

#         revenue_minor=[]
#         mask=(df3['Day']<=pivot_date) & (df3['Day']>=top_pivot_date)
#         sub=df3.loc[mask]

#         add_date=sub.iloc[0,2]
#         for i in range(ma_window):
#             print(i)
#             add_date=add_date+timedelta(days=1)
#             add_date=pd.to_datetime(add_date)    
#             sub2=sub[sub['Day']==add_date]
#             revenue_minor.append(np.mean(sub2['Revenue']))

#         start=0
#         end=len(revenue_minor)-2

#         while(end<ma_window+days_to_fix):
#             ma=np.mean(revenue_minor[start:end+1])
#             revenue_minor.append(ma)
#             start+=1
#             end+=1
#         revenue.append(revenue_minor)

In [109]:
# len(pre_active_campaign)

In [110]:
# revenue=[]
# ads=[]
# installs=[]
# impressions=[]
# installsp=[]
# clicks=[]
# organic=[]

# campaign_revenue(df2,revenue)
# campaign_ads(df2,ads)
# campaign_installs(df2,installs)
# campaign_impressions(df2,impressions)
# campaign_installsp(df2,installsp)
# campaign_clicks(df2,clicks)
# campaign_organic(df2,organic)

In [111]:
# fix_final=pd.DataFrame()

In [112]:
# len(impressions)

In [113]:
# for i in range(len(pre_active_campaign)):
#     print(i)
#     fix=df2[df2['Campaign']==pre_active_campaign[i]]
#     last_date=fix.iloc[fix.shape[0]-1,2]
#     last_date=pd.to_datetime(last_date)

#     pivot_date=last_date-timedelta(days=30)
#     pivot_date=pd.to_datetime(pivot_date)

#     add_date=pivot_date
#     add_date=pd.to_datetime(add_date)

#     fix_minor=fix[fix['Day']==add_date]
#     cond=len(impressions[i])
#     cond=cond-days_to_fix-1

#     for j in range(days_to_fix-1):
#         print(i,j)
#         fix_minor=fix[fix['Day']==add_date]

#         fix_minor['AS/Impressions']=impressions[i][cond]
#         fix_minor['AS/Installs']=installs[i][cond]
#         fix_minor['AS/Installs(paid)']=installsp[i][cond]
#         fix_minor['AS/Organic Installs']=organic[i][cond]
#         fix_minor['AS/Clicks']=clicks[i][cond]
#         fix_minor['Revenue']=revenue[i][cond]

#         fix_final=pd.concat([fix_final,fix_minor])

#         add_date=add_date+timedelta(days=1)
        
#         cond+=1
    

In [114]:
# fix_final.sort_values(['Day'])

In [115]:
# def impressions_predict():
#     for i in range(len(pre_active_campaign)):        
#         start=len(impressions[i])-1-prediction_ma_window
#         end=len(impressions[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(impressions[i][start:end+1])
#             impressions[i].append(ma)
#             start+=1
#             end+=1

In [116]:
# def revenue_predict():
#     for i in range(len(pre_active_campaign)):
#         start=len(revenue[i])-1-prediction_ma_window
#         end=len(revenue[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(revenue[i][start:end+1])
#             revenue[i].append(ma)
#             start+=1
#             end+=1

In [117]:
# def ads_predict():
#     for i in range(len(pre_active_campaign)):
#         start=len(ads[i])-1-prediction_ma_window
#         end=len(ads[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(ads[i][start:end+1])
#             ads[i].append(ma)
#             start+=1
#             end+=1

In [118]:
# def installs_predict():
#     for i in range(len(pre_active_campaign)):
#         start=len(installs[i])-1-prediction_ma_window
#         end=len(installs[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(installs[i][start:end+1])
#             installs[i].append(ma)
#             start+=1
#             end+=1

In [119]:
# def installsp_predict():
#     for i in range(len(pre_active_campaign)):
#         start=len(installsp[i])-1-prediction_ma_window
#         end=len(installsp[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(installsp[i][start:end+1])
#             installsp[i].append(ma)
#             start+=1
#             end+=1

In [120]:
# def clicks_predict():
#     for i in range(len(pre_active_campaign)):
#         start=len(clicks[i])-1-prediction_ma_window
#         end=len(clicks[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(clicks[i][start:end+1])
#             clicks[i].append(ma)
#             start+=1
#             end+=1

In [121]:
# def organic_predict():
#     for i in range(len(pre_active_campaign)):
#         start=len(organic[i])-1-prediction_ma_window
#         end=len(organic[i])-1
#         for j in range(days_to_predict):
#             ma=np.mean(organic[i][start:end+1])
#             organic[i].append(ma)
#             start+=1
#             end+=1

In [122]:
# impressions_predict()
# organic_predict()
# clicks_predict()
# installsp_predict()
# installs_predict()
# ads_predict()
# revenue_predict()

In [123]:
# # A[np.isnan(A)] = 0

# impressions=np.nan_to_num(impressions)
# organic=np.nan_to_num(organic)
# clicks=np.nan_to_num(clicks)
# installsp=np.nan_to_num(installsp)
# installs=np.nan_to_num(installs)
# ads=np.nan_to_num(ads)
# revenue=np.nan_to_num(revenue)
# # for i in range(len(pre_active_campaign)):
# #     one=impressions[i]
# #     one[np.isnan(one)]=0
# #     organic[np.isnan(organic)]=0
# #     clicks[np.isnan(clicks)]=0
# #     installsp[np.isnan(installsp)]=0
# #     installs[np.isnan(installs)]=0
# #     ads[np.isnan(ads)]=0
# #     revenue[np.isnan(revenue)]=0

In [124]:
# build=df2.copy()

In [125]:
# build=build.replace(float('inf'),0.0)
# build=build.fillna(0)

In [126]:
# build.columns

In [127]:
# build2=build.copy()

In [128]:
# cols_name=['Year', 'Month', 'Day', 'Country', 'Channel', 'OS name', 'Campaign','CTR','Ad Spend','Revenue', 'ROAS',
#        'DayNumber', 'AS/Impressions', 'AS/Installs', 'AS/Installs(paid)',
#        'AS/Organic Installs', 'AS/Clicks']

# cols=[0,1,2,3,4,5,6,8,11,20,21,22,23,24,25,26,27]

In [129]:
# build2=build2[build2.columns[cols]]

In [130]:
# build2

In [131]:
# len(revenue[0])

In [132]:
# nullcc=build2[build2['Country']==0].index
# build2=build2.drop(nullcc)

In [133]:
# build2

In [134]:
# X=build2.drop(['ROAS'],axis='columns')
# Y=build2['ROAS']

In [135]:
# X2=X.copy()

In [136]:
# from sklearn.preprocessing import LabelEncoder
# le=LabelEncoder()
# X['Campaign']=le.fit_transform(X['Campaign'])
# X['OS name']=le.fit_transform(X['OS name'])
# X['Country']=le.fit_transform(X['Country'])
# X['Channel']=le.fit_transform(X['Channel'])

In [137]:
# X=X.drop(['Day'],axis='columns')

In [138]:
# from xgboost import XGBRegressor
# xgb=XGBRegressor(objective='count:poisson',n_estimators=250)

# xgb.fit(X,Y)

In [139]:
# xgb.score(X,Y)

In [140]:
# Y_predicted=xgb.predict(X)

In [141]:
# df2['Predicted ROAS']=Y_predicted

In [142]:
# df2.to_csv('intermediate_results.csv')

In [143]:
# df2

Future Prediction

In [144]:
# final=pd.DataFrame()

In [145]:
# last_date=df2.iloc[df2.shape[0]-1,2]
# last_date=pd.to_datetime(last_date)

# prediction_window=30

# final=pd.DataFrame()

# c=0

In [146]:

# for j in range(days_to_predict):
# #run this cell twice to get proper values for dates
#     print(j)
#     test2=pd.DataFrame()
#     last_date=last_date+timedelta(days=1)

# #     test2['Year']=last_date.year
# #     test2['Month']=last_date.month

#     df2=df2.replace(float('inf'),0.0)
#     df2=df2.fillna(0)

#     nullcc=df2[df2['Country']==0].index
#     df2=df2.drop(nullcc)

#     cc=[]
#     for i in range(len(pre_active_campaign)):
#         sample=df2[df2['Campaign']==pre_active_campaign[i]]['Country'].unique()[0]
#         cc.append(sample)

#     os=[]
#     for i in range(len(pre_active_campaign)):
#         sample=df2[df2['Campaign']==pre_active_campaign[i]]['OS name'].unique()[0]
#         os.append(sample)

#     ctr=[]
#     for i in range(len(pre_active_campaign)):
#         sample=df2[df2['Campaign']==pre_active_campaign[i]]['CTR'].unique()[0]
#         ctr.append(sample)

#     channel=[]
#     for i in range(len(pre_active_campaign)):
#         sample=df2[df2['Campaign']==pre_active_campaign[i]]['Channel'].unique()[0]
#         channel.append(sample)

#     test2['Country']=cc
#     test2['Channel']=channel
#     test2['OS name']=os
#     test2['Campaign Name']=pre_active_campaign
#     test2['CTR']=ctr
    
#     c+=1

#     ads2=[]
#     impressions2=[]
#     installs2=[]
#     installsp2=[]
#     oragnic2=[]
#     clicks2=[]
#     revenue2=[]

#     for k in range(len(pre_active_campaign)):
# #         len(ads[j])-days_to_predict-1+i
#         ads2.append(ads[k][len(ads[k])-days_to_predict-1+i])
#         impressions2.append(impressions[k][len(impressions[k])-days_to_predict-1+i])
#         installs2.append(installs[k][len(installs[k])-days_to_predict-1+i])
#         installsp2.append(installsp[k][len(installsp[k])-days_to_predict-1+i])
#         oragnic2.append(organic[k][len(organic[k])-days_to_predict-1+i])
#         clicks2.append(clicks[k][len(clicks[k])-days_to_predict-1+i])
#         revenue2.append(clicks[k][len(clicks[k])-days_to_predict-1+i])

#     test2['Ad Spend']=ads2
#     test2['Revenue']=revenue2
#     test2['DayNumber']=last_date.day

#     test2['AS/Impressions']=impressions2
#     test2['AS/Installs']=installs2
#     test2['AS/Installs(paid)']=installsp2
#     test2['AS/Organic Installs']=oragnic2
#     test2['AS/Clicks']=clicks2
    
#     test2['Year']=last_date.year
#     test2['Month']=last_date.month
    
#     test2=test2[['Year', 'Month', 'Country', 'Channel', 'OS name', 'Campaign Name',
#        'CTR', 'Ad Spend', 'Revenue', 'DayNumber', 'AS/Impressions',
#        'AS/Installs', 'AS/Installs(paid)', 'AS/Organic Installs', 'AS/Clicks']]
#     # for i in range(len(pre_active_campaign)): 
#     #     test2['Impressions/AS'][i]=impressions[i][5]
#     #     test2['Installs/AS'][i]=installs[i][5]
#     #     test2['Installs(paid)/AS'][i]=installsp[i][5]
#     #     test2['Organic Installs/AS'][i]=oragnic[i][5]
#     #     test2['Clicks/AS'][i]=clicks[i[5]]
    
#     final=pd.concat([test2,final])
    
#     final=final.reset_index(drop=True)


In [147]:
# len(pre_active_campaign)

In [148]:
# final

In [149]:
# final["AS/Clicks"].unique()

In [150]:
# final=final.fillna(0)

In [151]:
# test3=final.copy()

In [152]:
# final['Campaign Name']=le.fit_transform(final['Campaign Name'])
# final['OS name']=le.fit_transform(final['OS name'])
# final['Country']=le.fit_transform(final['Country'])
# final['Channel']=le.fit_transform(final['Channel'])

In [153]:
# final

In [154]:
# final=final.sort_values(['Month','DayNumber'])

In [155]:
# final=final.reset_index(drop=True)

In [156]:
# final

In [157]:
# roas_predicted=xgb.predict(final)
# roas_predicted

In [158]:
# test3=test3.sort_values(['Month','DayNumber'])
# test3=test3.reset_index(drop=True)

In [159]:
# test3['Predicted ROAS']=roas_predicted

In [160]:
# test3['Predicted ROAS']

In [161]:
# test3

In [162]:
# test3.to_csv("model_6_results.csv")

In [163]:
# test3.columns